<h2><b>Generate mood labels for the main dataset</b></h2>

<h4>Import necessary libraries</h4>

In [1]:
import sklearn
import pandas as pd
from numpy import mean
from numpy import std
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from matplotlib import pyplot as plt

<h3>Load Labelled data</h3>

In [2]:
data = pd.read_csv('data_moods.csv')
data.columns

Index(['name', 'album', 'artist', 'id', 'release_date', 'popularity', 'length',
       'danceability', 'acousticness', 'energy', 'instrumentalness',
       'liveness', 'valence', 'loudness', 'speechiness', 'tempo', 'key',
       'time_signature', 'mood'],
      dtype='object')

In [3]:
X = data.select_dtypes('number')
y = data['mood']

<h3>Select Number of features</h3>

In [11]:
import os

def gen_models(model_name):
	"""
	input: a model name
	output: a dictionary of models with number of features in range [5,9]
	"""
	models = dict()
	for i in range(2, 10):
		rfe = RFE(estimator=model_name(), n_features_to_select=i)
		model = DecisionTreeClassifier()
		models[str(i)] = Pipeline(steps=[('s',rfe),('m',model)])
	return models



def evaluate_model(model, X, y):
    """
    Runs 3 fold cross validation using provided model, X and y
    Returns scores
    """
    cross_validation = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cross_validation, n_jobs=-1, error_score='raise')
    return scores



def plot_model_comparisons(model_names):
    """
    input: array of model names as defined in sklearn library
    output: plots a 2X2 grid of plots, so input array length is expected to be 4
    """
    assert(len(model_names) == 4)
    fig, axs = plt.subplots(2, 2, figsize=(10, 8))
    axs = axs.flatten()

    for i, ax in enumerate(axs):
        model_name = model_names[i]
        models = gen_models(model_name)
        results, names = [], []
        for name, model in models.items():
            scores = evaluate_model(model, X, y)
            results.append(scores)
            names.append(name)
            print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
        
        ax.boxplot(results, labels=names, showmeans=True)
        ax.set_title(str(model_name).split('.')[-1])

    plt.tight_layout()
    plt.savefig(os.path.join(os.getcwd(), 'model_comparisons.png'))

In [12]:
%%capture

import warnings
from sklearn.exceptions import ConvergenceWarning

# Suppress ConvergenceWarnings
warnings.filterwarnings('ignore', category=ConvergenceWarning)

model_names = [DecisionTreeClassifier, LogisticRegression, Perceptron, GradientBoostingClassifier]
plot_model_comparisons(model_names=model_names)

/Users/washikuddinahmedmollah/Documents/CS470_IntroToAi/project2/cs470_introAi_final_project_music_recommendation/myenv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/washikuddinahmedmollah/Documents/CS470_IntroToAi/project2/cs470_introAi_final_project_music_recommendation/myenv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://sci

In [13]:
big_data = pd.read_csv('data.csv')
big_data.columns

Index(['valence', 'year', 'acousticness', 'artists', 'danceability',
       'duration_ms', 'energy', 'explicit', 'id', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'name', 'popularity', 'release_date',
       'speechiness', 'tempo'],
      dtype='object')

In [53]:
big_data.select_dtypes('number').columns

Index(['valence', 'year', 'acousticness', 'danceability', 'duration_ms',
       'energy', 'explicit', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'popularity', 'speechiness', 'tempo'],
      dtype='object')

<h3>Loading data again</h3>

In [62]:
X = data.select_dtypes('number')
y = data['mood']
X = X.drop(columns=['length', 'time_signature'])

In [63]:
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=9)
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s', rfe), ('m', model)])
pipeline.fit(X, y)

Pipeline(steps=[('s',
                 RFE(estimator=DecisionTreeClassifier(),
                     n_features_to_select=9)),
                ('m', DecisionTreeClassifier())])

In [64]:
chosen_features = ['popularity','danceability', 'acousticness', 'energy',
       'instrumentalness', 'liveness', 'valence', 'loudness', 'speechiness',
       'tempo', 'key']

predictions = pipeline.predict(big_data[chosen_features])

# Create a new column in df_unlabeled to store the predictions
big_data['predicted_mood'] = predictions

In [65]:
big_data.predicted_mood.unique()

array(['Calm', 'Sad', 'Happy', 'Energetic'], dtype=object)

In [66]:
big_data.shape

(170653, 21)

In [59]:
big_data.to_csv('data.csv', index=False)

In [60]:
big_data['predicted_mood'].value_counts()

predicted_mood
Sad          83891
Happy        44436
Energetic    23104
Calm         19222
Name: count, dtype: int64